Пытаемся реализовать парсинг сайта cian.ru. Нами был выбран город *Махачкала*, собираем информацию только про те квартиры, которые сдаются посуточно.



In [1]:
from bs4 import BeautifulSoup # библиотека для анализа HTML-кода WEB-страниц
from selenium import webdriver # библиотека для работы с веб драйверами( уменьшения проблем свзанных с множественными переходами на сайты) (с ней происходит меньше проблем при наличии защиты от ддос атак)



def parsCian(page):
    '''Функция сбора ссылок всех квартир с каждой страницы сайта'''
    currPage = "https://mahachkala.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=" + str(page)  +"&region=4857&type=1"
    driver = webdriver.Chrome() #запускаем пакет селениума и драйвер браузера
    driver.get(currPage) #переходим на страницу
    data = driver.page_source #получаем код страницы
    soup = BeautifulSoup(data, "html.parser") #передаем страничку - выйдет html-код странички
    links = []
    
    for i in soup.find_all("a", {"class": "_93444fe79c--link--eoxce"}): #извлекаем все URL-адреса, найденные на странице в тегах <a>
        if i:
            links.append(i.get("href")) #возвращаем и сохраняем в links значение ссылки

    return links

allLinks = []
for i in range(1,60,1): #парсим 60 страниц, выбран такой интервал, чтобы в итоге получилось большее 1000 наблюдений
    allLinks += parsCian(i)

Собираем все странички квартир. Удаляем те, которые сильно отличаются от остальных (мало информации или сильно отличающаяся информация от большинства страниц).

In [2]:
from itertools import groupby

#удаляем ссылки которые не подходят из-за сильных различий в структуре html и некоторых другие метки 
allLinksFiltered = [el for el, _ in groupby(allLinks)]
allLinksFiltered2 = []
for i in allLinksFiltered:
    if("cian.tvil.ru" not in i):
        allLinksFiltered2.append(i)


В функции собираем интересующие нас метки со страничек каждой квартиры.
Выбраны метки, чаще всего встречающиеся на страничках, которые, по нашему мнению, наиболее важны для заселяющихся и с которыми дальше можно будет что-то придумать.

In [4]:
from fake_useragent import UserAgent #будем имитировать реального пользователя с помощью этого пакета
from selenium.webdriver.chrome.options import Options #добавляем опции хрома


In [ ]:



def parsLink(link):
    
    options = Options()
    options.add_argument('--headless'); # отключение открытия на компьютере нового браузера для каждой страницы
    ua = UserAgent()
    userAgent = ua.random  #генерация случайного "пользователя" для каждой страницы(для обхода защиты)
    options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Chrome(chrome_options=options) #использование выбранных выше опций при парсинге
    driver.get(str(link))
    data = driver.page_source
    soup = BeautifulSoup(data, "html.parser")
    
    #Получаем цену
    try:
        price = soup.find_all('span', {'itemprop' : 'price'}) #достаем данные о цене
        priceValue = price[0].text
        priceArr = priceValue.split("\xa0") #разделяем цену от валюты ("\xa0" неразрывный пробел)
        price = priceArr[1] + priceArr[2] # соединяем тысячи и остальные цифры
    except:
        price = 0
    
    #получаем агенство
    try:
        agency = soup.find_all('div', {'class' : 'a10a3f92e9--title--YaRYv'})[0].text 
        
    except:
        agency = 0
        
    #общая площадь
    try:
        total = soup.find_all('div', {'class' : 'a10a3f92e9--info-value--bm3DC'})[0].text.split("\xa0")[0] #общая площадь и жилая имели одинаковые теги, поэтому получаем массив элементов, 1-й элемент массива общая, 2-й жилая
        if("из" in total):
            total = 0
        
    except:
        total = 0
    
    #жилая
    try:
        living = soup.find_all('div', {'class' : 'a10a3f92e9--info-value--bm3DC'})[1].text.split("\xa0")[0]
        if("из" in living):
            living = 0
        
    except:
        living = 0
    
    #этаж
    try:
        floor = 0
        floorText = soup.find_all('div', {'class' : 'a10a3f92e9--info-value--bm3DC'})
        for i in floorText:
            if("из" in i.text):
                floor = i.text.split(" ")[0]#этаж содержался в формате 4 из 10, поэтому достаем 1-ю цифру из записи для текущего этажа
        
    except:
        floorText = 0
        floor = 0
    
    #всего этажей
    
    try:
        totalFloor = 0
        floorText = soup.find_all('div', {'class' : 'a10a3f92e9--info-value--bm3DC'})
        for i in floorText:
            if("из" in i.text):
                totalFloor = i.text.split(" ")[2] #так же как и в прошлом пункте, но берем последнюю
    except:
        totalFloor = 0
    
    
    
    #Дети
    kids = 0
    try:
        kids = soup.find_all('li', {'class' : 'a10a3f92e9--item--DCJ3N a10a3f92e9--kids--hABK5'})
        if(kids[0].text == "Можно с детьми"):
            kids = 1
    except:
        kids = 0
    
    #фурнитура
    
    try:
        features = soup.find_all('ul', {'class' : 'a10a3f92e9--container--P4zGu'})[-1].text #получаем список фурнитуры и проверям наличие по вхождению
        fridge = 0 
        dishwasher = 0
        washmashine = 0
        roomfurniture = 0
        kitchenfurniture = 0
        TV = 0
        net = 0
        condition = 0
        dush = 0
        vanna = 0
        if 'Холодильник' in features:
            fridge = 1
        if 'Посудомоечная машина' in features:
            dishwasher = 1
        if 'Стиральная машина' in features:
            washmashine = 1
        if 'Мебель в комнатах' in features:
            roomfurniture = 1
        if 'Мебель на кухне' in features:
            kitchenfurniture = 1
        if 'Телевизор' in features:
            TV = 1
        if 'Интернет' in features:
            net = 1
        if 'Кондиционер' in features:
            condition = 1
        if 'Душевая кабина' in features:
            dush = 1
        if 'Ванна' in features:
            vanna = 1
    except:
        fridge = 0 
        dishwasher = 0
        washmashine = 0
        roomfurniture = 0
        kitchenfurniture = 0
        TV = 0
        net = 0
        condition = 0
        dush = 0
        vanna = 0
    
    
    #aдрес
    district = 0
    street = 0
    fulladdress = 0
    try:
        fulladdress = soup.find_all('a', {'class' : 'a10a3f92e9--link--ulbh5 a10a3f92e9--address-item--ScpSN'})#получаем список данных о адресе(имеют только общие теги)
        for i in fulladdress:
            if("р-н" in i.text or "район" in i.text): #проверяем не является ли районом элемент
                district = i.text.replace("р-н", "").replace("район","") #меняем сокращения слова район на слово район для однородности
            if("ул" in i.text or "туп" in i.text):#проверяем не является ли улицей элемент
                street = i.text.replace("ул.", "").replace("улица", "").replace("туп.", "")#делаем как с районом
    except:
        fulladdress = 0
        resp = 0
        district = 0
        street = 0
        house = 0
    
    
    ans = [link, price, agency, total, living, floor, totalFloor, kids, fridge,
          dishwasher, washmashine, roomfurniture, kitchenfurniture, TV, net, 
          condition, dush, vanna, district, street] #собираем все вместе
    return ans #возвращаем данные о квартире
# В data собираем данные с каждой ссылочки через прогон функции parsLink
data = []
for i in allLinksFiltered2:
    data.append(parsLink(i)) #собираем данные вместе

C:\Users\Тапок\AppData\Local\Temp\ipykernel_1812\4240956828.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options) #использование выбранных выше опций при парсинге


Итоговый датафрейм, который дальше будет редачиться и юзаться.

In [ ]:
import pandas as pd
import numpy as np

df =pd.DataFrame(data = data, columns =  ["ссылка", "цена", "агенство", "общая площадь", "жилая площадь", "этаж",
             "всего этажей", "Дети", "холодильник", "посудомойка", "стиральная машина",
             "мебель в комнатах", "мебель на кухне", "телевизор", "интернет", "кондиционер",
             "душ", "ванна", "район", "улица"]) #задаем параметры для таблицы и заполняем ее данными


df.style.hide_index()
df 

Записываем в файл.

In [ ]:
df.to_csv(r'C:\Users\murad\OneDrive\Рабочий стол\dataframe.csv', sep=',', encoding='utf-8')

In [ ]:
#удалили после чекпоинта все технические принты и пипы, оставили только вывод датафрейма